In [7]:
import pandas as pd 
import numpy as np
import yfinance as yf
import math
from scipy import stats

In [8]:
tickers = pd.read_csv('top_50_indian_stocks.csv')
tickers.head()

,Ticker,Company Name
0,RELIANCE.NS,Reliance Industries
1,TCS.NS,Tata Consultancy Services
2,HDFCBANK.NS,HDFC Bank
3,INFY.NS,Infosys
4,ICICIBANK.NS,ICICI Bank


## Weighted Scoring model for finding the final score

In [12]:
def create_dividend_df(tickers):
    columns = [
        "Ticker", 
        "Dividend Yield(%)", 
        "Dividend Rate",
        "Payout Ratio(%)", 
        "5 Year Avg Dividend Yield(%)", 
        "Earning Growth(%)"
        
    ]
    
    dividend_df = pd.DataFrame(columns=columns)
    
    for stock in tickers:
        ticker = yf.Ticker(stock)
        info = ticker.info 
        
        dividend_yield = info.get("dividendYield", np.nan) * 100 if info.get("dividendYield") else np.nan
        dividend_rate = info.get("dividendRate", np.nan) 
        payout_ratio = info.get("payoutRatio", np.nan) * 100 if info.get("payoutRatio") else np.nan
        five_year_avg_dividend_yield = info.get("fiveYearAvgDividendYield", np.nan) * 100 if info.get("fiveYearAvgDividendYield") else np.nan
        earning_growth = info.get("earningsGrowth", np.nan) * 100 if info.get("earningsGrowth") else np.nan
        
        dividend_df.loc[len(dividend_df)] = [stock, dividend_yield, dividend_rate, payout_ratio, five_year_avg_dividend_yield, earning_growth]
        
    ## we have got our df with dividend information
    
    numeric_cols = [
          "Dividend Yield(%)", 
        "Dividend Rate",
        "Payout Ratio(%)", 
        "5 Year Avg Dividend Yield(%)", 
        "Earning Growth(%)"
    ]
    

    for col in numeric_cols:
        if col == "Payout Ratio":
            dividend_df[col+ " Normalised"] = 1 - ((dividend_df[col] - dividend_df[col].min()) / (dividend_df[col].max() - dividend_df[col].min()))
        else:
            dividend_df[col+ " Normalised"] = (dividend_df[col] - dividend_df[col].min()) / (dividend_df[col].max() - dividend_df[col].min())
        
    return dividend_df

In [13]:
tickers_list = tickers["Ticker"].values.tolist()
dividend_df = create_dividend_df(tickers_list)
dividend_df

,Ticker,Dividend Yield(%),Dividend Rate,Payout Ratio(%),5 Year Avg Dividend Yield(%),Earning Growth(%),Dividend Yield(%) Normalised,Dividend Rate Normalised,Payout Ratio(%) Normalised,5 Year Avg Dividend Yield(%) Normalised,Earning Growth(%) Normalised
0,RELIANCE.NS,41.0,5.50,8.300000,44.0,78.4,0.042403,0.033582,0.046889,0.042938,0.651948
1,TCS.NS,198.0,61.00,52.140000,132.0,6.0,0.227326,0.447761,0.357878,0.142373,0.338528
2,HDFCBANK.NS,116.0,11.00,25.150000,67.0,-2.2,0.130742,0.074627,0.166418,0.068927,0.303030
3,INFY.NS,429.0,63.00,65.900004,218.0,5.6,0.499411,0.462687,0.455487,0.239548,0.336797
4,ICICIBANK.NS,79.0,11.00,13.600000,99.0,14.6,0.087161,0.074627,0.084486,0.105085,0.375758
5,HINDUNILVR.NS,180.0,48.00,93.600000,154.0,5.6,0.206125,0.350746,0.651983,0.167232,0.336797
6,SBIN.NS,198.0,15.90,17.860001,139.0,9.7,0.227326,0.111194,0.114705,0.150282,0.354545
7,BAJFINANCE.NS,50.0,4.40,15.680000,24.0,19.7,0.053004,0.025373,0.099241,0.020339,0.397835
8,BHARTIARTL.NS,85.0,16.00,13.570000,80.0,41.8,0.094229,0.111940,0.084273,0.083616,0.493506
9,ITC.NS,383.0,15.70,90.029997,403.0,2.7,0.445230,0.109701,0.626658,0.448588,0.324242


In [16]:
tickers_list = tickers['Ticker'].values.tolist()

df = create_dividend_df(tickers_list)
numeric_cols = [
          "Dividend Yield(%)", 
        "Dividend Rate",
        "Payout Ratio(%)", 
        "5 Year Avg Dividend Yield(%)", 
        "Earning Growth(%)"
    ]


for col in numeric_cols:
 df[col]=df[col].fillna(df[col].mean())
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 0 to 49
Data columns (total 11 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Ticker                                   50 non-null     object 
 1   Dividend Yield(%)                        50 non-null     float64
 2   Dividend Rate                            50 non-null     float64
 3   Payout Ratio(%)                          50 non-null     float64
 4   5 Year Avg Dividend Yield(%)             50 non-null     float64
 5   Earning Growth(%)                        50 non-null     float64
 6   Dividend Yield(%) Normalised             48 non-null     float64
 7   Dividend Rate Normalised                 48 non-null     float64
 8   Payout Ratio(%) Normalised               46 non-null     float64
 9   5 Year Avg Dividend Yield(%) Normalised  46 non-null     float64
 10  Earning Growth(%) Normalised             49 non-null     

In [ ]:
weights = {
        "Dividend Yield(%) Normalised": 0.3,
        "Dividend Rate Normalised": 0.2,
        "Payout Ratio(%) Normalised": 0.2,
        "5 Year Avg Dividend Yield(%) Normalised": 0.2,
        "Earning Growth(%) Normalised": 0.1
    }

dividend_df["Dividend Score"] = dividend_df[[col for col in weights.keys()]].mul(list(weights.values())).sum(axis = 1)
dividend_df

,Ticker,Dividend Yield(%),Dividend Rate,Payout Ratio(%),5 Year Avg Dividend Yield(%),Earning Growth(%),Dividend Yield(%) Normalised,Dividend Rate Normalised,Payout Ratio(%) Normalised,5 Year Avg Dividend Yield(%) Normalised,Earning Growth(%) Normalised,Dividend Score
30,COALINDIA.NS,854.0,32.00,49.000000,891.0,-20.2,1.000000,0.231343,0.335603,1.000000,0.225108,0.635900
42,HEROMOTOCO.NS,256.0,130.00,55.630000,311.0,63.1,0.295642,0.962687,0.382635,0.344633,0.585714,0.485255
28,TECHM.NS,304.0,45.00,142.660000,218.0,34.0,0.352179,0.328358,1.000000,0.239548,0.459740,0.465209
20,ONGC.NS,631.0,14.75,47.100000,515.0,-1.4,0.737338,0.102612,0.322125,0.575141,0.306494,0.451826
13,HCLTECH.NS,371.0,54.00,86.300004,318.0,-7.7,0.431095,0.395522,0.600199,0.352542,0.279221,0.426903
3,INFY.NS,429.0,63.00,65.900004,218.0,5.6,0.499411,0.462687,0.455487,0.239548,0.336797,0.415047
9,ITC.NS,383.0,15.70,90.029997,403.0,2.7,0.445230,0.109701,0.626658,0.448588,0.324242,0.402983
22,POWERGRID.NS,427.0,11.75,65.639997,504.0,-2.5,0.497055,0.080224,0.453643,0.562712,0.301732,0.398605
31,BPCL.NS,324.0,10.00,38.210000,553.0,140.8,0.375736,0.067164,0.259062,0.618079,0.922078,0.393790
45,IOC.NS,220.0,3.00,57.100000,719.0,93.4,0.253239,0.014925,0.393062,0.805650,0.716883,0.390388


In [18]:
dividend_df = dividend_df.sort_values(by = "Dividend Score", ascending = False)
dividend_df.head(10)

,Ticker,Dividend Yield(%),Dividend Rate,Payout Ratio(%),5 Year Avg Dividend Yield(%),Earning Growth(%),Dividend Yield(%) Normalised,Dividend Rate Normalised,Payout Ratio(%) Normalised,5 Year Avg Dividend Yield(%) Normalised,Earning Growth(%) Normalised,Dividend Score
30,COALINDIA.NS,854.0,32.00,49.000000,891.0,-20.2,1.000000,0.231343,0.335603,1.000000,0.225108,0.635900
42,HEROMOTOCO.NS,256.0,130.00,55.630000,311.0,63.1,0.295642,0.962687,0.382635,0.344633,0.585714,0.485255
28,TECHM.NS,304.0,45.00,142.660000,218.0,34.0,0.352179,0.328358,1.000000,0.239548,0.459740,0.465209
20,ONGC.NS,631.0,14.75,47.100000,515.0,-1.4,0.737338,0.102612,0.322125,0.575141,0.306494,0.451826
13,HCLTECH.NS,371.0,54.00,86.300004,318.0,-7.7,0.431095,0.395522,0.600199,0.352542,0.279221,0.426903
3,INFY.NS,429.0,63.00,65.900004,218.0,5.6,0.499411,0.462687,0.455487,0.239548,0.336797,0.415047
9,ITC.NS,383.0,15.70,90.029997,403.0,2.7,0.445230,0.109701,0.626658,0.448588,0.324242,0.402983
22,POWERGRID.NS,427.0,11.75,65.639997,504.0,-2.5,0.497055,0.080224,0.453643,0.562712,0.301732,0.398605
31,BPCL.NS,324.0,10.00,38.210000,553.0,140.8,0.375736,0.067164,0.259062,0.618079,0.922078,0.393790
45,IOC.NS,220.0,3.00,57.100000,719.0,93.4,0.253239,0.014925,0.393062,0.805650,0.716883,0.390388
